In [28]:
import pickle
import os
import sys
import datetime
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
from sklearn.metrics import roc_auc_score    
from sklearn.cross_validation import StratifiedKFold

In [29]:
import nltk
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Read in datafile

In [30]:
# Read data from files
df = pd.read_csv('Reviews.csv')

In [31]:
df['Text'].loc[0]

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [32]:
print len(df)
print df["Text"].size

568454
568454


In [33]:
df.columns = ['Id', 'ProductId', 'UserId', "ProfileName", 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'label', 'Time', 'title', 'abstract']

In [34]:
df[0:2]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,label,Time,title,abstract
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


### Check label distribution-- almost all reviews have a score of 5...

In [35]:
#we are concerned about rating
print len(df.loc[df['label'] == 5])
print len(df.loc[df['label'] == 4])
print len(df.loc[df['label'] == 3])
print len(df.loc[df['label'] == 2])
print len(df.loc[df['label'] == 1])
print len(df.loc[df['label'] == 0])

363122
80655
42640
29769
52268
0


In [36]:
#randomly subset those with label 5 to be more similar to actual amount (*** and most importantly to reduce size of dataset)
df_5 = df.loc[df['label'] == 5]
df_5 = df_5.sample(n=50000, random_state=10)

In [37]:
print len(df_5)

50000


In [38]:
print len(df.loc[df['label'] != 5])
df_allElse = df.loc[df['label'] != 5]

205332


In [39]:
df = pd.concat([df_allElse, df_5])
print len(df)

255332


In [40]:
print len(df.loc[df['label'] == 5])
print len(df.loc[df['label'] == 4])
print len(df.loc[df['label'] == 3])
print len(df.loc[df['label'] == 2])
print len(df.loc[df['label'] == 1])
print len(df.loc[df['label'] == 0])

50000
80655
42640
29769
52268
0


### Apply Word2Vec Inversion

In [41]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [42]:
def buildModels(train1, test1, docs, basemodel, label):
    
    #add boolean labels
    train1['boolean_label'] = 1
    for row in train1.iterrows():
        if row[1]['label'] != label:
            train1.set_value(row[0], 'boolean_label', 0)
            
    test1['boolean_label'] = 1
    for row in test1.iterrows():
        if row[1]['label'] != label:
            test1.set_value(row[0], 'boolean_label', 0)
    
    # ****** Split the training set into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    #here change to include all journal name labels of positive and negative
    inxs_pos = np.where(train1['boolean_label'] == 1)[0].tolist()
    inxs_neg = np.where(train1['boolean_label'] == 0)[0].tolist()

    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
         
    #train models
    models = [deepcopy(basemodel) for y in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )
    
    probs = docprob(docs,models).astype(object)
    
    predictions = np.ones((probs.shape[0]))

    predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

    score = roc_auc_score(test1["boolean_label"], predictions)
    
    return score

In [45]:
a = datetime.datetime.now().replace(microsecond=0)

y = df['label'].values

#set random state
skf = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=10)

i = 1
avg = []
avg1 = []
avg2 = []
avg3 = []
avg4 = []
avg5 = []
for train_index, test_index in skf:

    print("ROUND", i)
    i = i + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]

    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences = []  # Initialize an empty list of sentences

    for j in range(len(train1)):
        review = train1['abstract'].iloc[j]
        sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

    print "Building and training w2v models"
    ## create a w2v learner 
    basemodel = Word2Vec(
        workers=multiprocessing.cpu_count(), # use your cores
        iter=3, # iter = sweeps of SGD through the data; more is better
        hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
    basemodel.build_vocab(sentences) 

    print "Parsing test sentences"
    # read in the test set as a list of a list of words

    docs = []
    for k in range(len(test1)):
        review = test1["abstract"].iloc[k]
        sents = KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        docs.append(sents)

    print "scoring test set"
    tup1 = buildModels(train1, test1, docs, basemodel, 1)
    avg1.append(tup1)
    avg.append(tup1)
    
    tup2 = buildModels(train1, test1, docs, basemodel, 2)
    avg2.append(tup2)
    avg.append(tup2)

    tup3 = buildModels(train1, test1, docs, basemodel, 3)
    avg3.append(tup3)
    avg.append(tup3)

    tup4 = buildModels(train1, test1, docs, basemodel, 4)
    avg4.append(tup4)
    avg.append(tup4)

    tup5 = buildModels(train1, test1, docs, basemodel, 5)
    avg5.append(tup5)
    avg.append(tup5)

print("average of Score 1 model:", sum(avg1)/float(len(avg1)))
print("average of Score 2 model:", sum(avg2)/float(len(avg2)))
print("average of Score 3 model:", sum(avg3)/float(len(avg3)))
print("average of Score 4 model:", sum(avg4)/float(len(avg4)))
print("average of Score 5 model:", sum(avg5)/float(len(avg5)))
print ""
print("average across all models:", sum(avg)/float(len(avg)))

    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: ".." looks like a filename, not markup. You should probably open thi

Building and training w2v models
Parsing test sentences


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "///" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)


scoring test set


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('ROUND', 2)
Building and training w2v models
Parsing test sentences
scoring test set
('ROUND', 3)
Building and training w2v models
Parsing test sentences
scoring test set
('ROUND', 4)
Building and training w2v models
Parsing test sentences
scoring test set
('ROUND', 5)
Building and training w2v models
Parsing test sentences
scoring test set
('average of Score 1 model:', 0.8191279999723371)
('average of Score 2 model:', 0.66959156323702385)
('average of Score 3 model:', 0.69183501796552815)
('average of Score 4 model:', 0.73211042386956982)
('average of Score 5 model:', 0.74585413800471967)

('average across all models:', 0.73170382860983585)
3:16:44


In [44]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences